In [ ]:
!pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 11.5 MB/s eta 0:00:00


In [ ]:
!pip install qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.5 MB/s eta 0:00:00


In [ ]:
import networkx as nx

In [ ]:
import qiskit
import numpy as np
from qiskit.providers.fake_provider import *
import networkx as nx
from scipy.optimize import minimize
from qiskit import QuantumCircuit, execute, Aer, QuantumRegister
import matplotlib.pyplot as plt

In [ ]:
# 3-regular graph with six nodes
G = nx.random_regular_graph(d=3, n=6, seed=12345)

In [ ]:
# Hamiltonian layer
def zz_term(qc, q1, q2, gamma):
    qc.cx(q1,q2)
    qc.rz(2*gamma, q2)
    qc.cx(q1,q2)

def get_cost_circuit(qc, G, gamma):
    N = G.number_of_nodes()
    for i, j in G.edges():
        zz_term(qc, i, j, gamma)
    return qc

In [ ]:
# Mixer layer
def x_term(qc, q1, beta):
    qc.rx(2*beta, q1)


def get_mixer_circuit(qc, G, beta):
    N = G.number_of_nodes()
    for n in G.nodes():
        x_term(qc, n, beta)
    return qc

In [ ]:
# Combine two layers and get the QAOA circuit
def get_qaoa_circuit(G, beta, gamma):
    p = 1 # infering number of QAOA steps from the parameters passed
    N = G.number_of_nodes()
    qc = QuantumCircuit(N,N)
    # get the initial state
    qc.h(range(N))
    # second, apply p alternating operators
    for i in range(p):
        qc = get_cost_circuit(qc,G,gamma[i])
        qc = get_mixer_circuit(qc,G,beta)
    # finally, do not forget to measure the result!
    qc.barrier(range(N))
    qc.measure(range(N), range(N))
    return qc

def measure_circuit(qc, n_trials=1024):
    """Get the output from circuit, either measured samples or full state vector"""
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=n_trials)
    result = job.result()
    bitstrings = invert_counts(result.get_counts())
    return bitstrings

In [ ]:
# define the objective function
def invert_counts(counts):
    return {k[::-1]:v for k, v in counts.items()}

def maxcut_obj(x,G):
    ''' calculate energy from one bitstring '''
    cut = 0
    for i, j in G.edges():
        if x[i] != x[j]:
            # the edge is cut
            cut -= 1
    return cut

def compute_maxcut_energy(counts, G):
    ''' calculate energy from samples '''
    energy = 0
    total_counts = 0
    for meas, meas_count in counts.items():
        obj_for_meas = maxcut_obj(meas, G)
        energy += obj_for_meas * meas_count
        total_counts += meas_count
    return energy / total_counts

def get_black_box_objective(G,p):
    backend = Aer.get_backend('qasm_simulator')
    def f(theta):
        # first half is betas, second half is gammas
        beta = theta[:p][0]
        gamma = theta[p:]
        ## define the qaoa circuit
        qc = get_qaoa_circuit(G,beta, gamma)
        counts = measure_circuit(qc, n_trials=1024)
        # return the energy
        return compute_maxcut_energy(counts, G)
    return f

In [ ]:
seed_value = 42
np.random.seed(seed_value)

In [ ]:
p = 1
obj = get_black_box_objective(G, p)
init_point = np.random.rand(p*2)

# Training the QAOA
result = minimize(obj, init_point, method='COBYLA', options={'maxiter':200, 'disp': True})
result

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -6.224609375
       x: [ 1.186e+00  1.281e+00]
    nfev: 29
   maxcv: 0.0

In [ ]:
print(G.number_of_nodes())

6


In [ ]:
from operator import itemgetter
n_node = 6
def hilbert_iter(n_qubits):
    """
    An iterator over all 2**n_qubits bitstrings in a given hilbert space basis.
    """
    for n in range(2**n_qubits):
        yield np.fromiter(map(int, np.binary_repr(n, width=n_qubits)), dtype=np.bool, count=n_qubits)

all_config  = 1*np.array(list(hilbert_iter(n_node)), dtype=np.bool_)
best_cut, best_solution = min([(maxcut_obj(x,G),x) for x in all_config], key=itemgetter(0))
print(f"Best string: {best_solution} with cut: {-best_cut}")

Best string: [0 1 1 1 0 0] with cut: 9


<ipython-input-22-2f6e3892e511>:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  yield np.fromiter(map(int, np.binary_repr(n, width=n_qubits)), dtype=np.bool, count=n_qubits)
